In [114]:
import sys
print("현재 커널 파이썬:", sys.executable)

현재 커널 파이썬: /usr/bin/python3.11


In [115]:
#!{sys.executable} -m pip install transformers bert-score peft accelerate

In [116]:
# !pip install bert-score

In [117]:
# !pip install peft

In [118]:
# !pip install hf_transfer

In [119]:
#!{sys.executable} -m pip install -U hf_transfer

In [120]:
import sys
print(sys.executable)  # 지금 쓰는 파이썬 경로 확인

!{sys.executable} -m pip show hf_transfer

/usr/bin/python3.11


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Name: hf_transfer
Version: 0.1.9
Summary: Speed up file transfers with the Hugging Face Hub.
Home-page: 
Author: 
Author-email: 
License: 
Location: /usr/local/lib/python3.11/dist-packages
Requires: 
Required-by: 


In [121]:
import hf_transfer
print(hf_transfer.__version__)

0.1.9


In [122]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from bert_score import score
from peft import AutoPeftModelForCausalLM
import torch

In [123]:
adapter_model_name = "poketmon/ax-trained-model-v3-2"

base_model_name = "skt/A.X-4.0-Light"

In [124]:
base_tokenizer = AutoTokenizer.from_pretrained(base_model_name)
base_model = AutoModelForCausalLM.from_pretrained(base_model_name, torch_dtype="auto", device_map="auto", trust_remote_code=True)
base_pipe = pipeline("text-generation", model=base_model, tokenizer=base_tokenizer, return_full_text=False)

Loading checkpoint shards: 100%|██████████| 3/3 [00:03<00:00,  1.26s/it]
Device set to use cuda:0


In [125]:
print(f"🚀 베이스 모델 로드 중... (base={base_model_name})")
tokenizer = AutoTokenizer.from_pretrained(
    base_model_name,
    trust_remote_code=True   
)

print(f"🚀 LoRA 어댑터 로드 중... (adapter={adapter_model_name})")
model = AutoPeftModelForCausalLM.from_pretrained(
    adapter_model_name,
    dtype=torch.float16,     
    trust_remote_code=True
)

if torch.cuda.is_available():
    model = model.to("cuda:0")
    device_for_pipe = 0
    print("✅ 모델을 cuda:0 에 올렸습니다.")
else:
    device_for_pipe = -1           # GPU 없으면 CPU
    print("⚠️ CUDA를 사용할 수 없어 CPU로 동작합니다.")

print("🚀 pipeline 생성 중...")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0,
    return_full_text=False, 
)

🚀 베이스 모델 로드 중... (base=skt/A.X-4.0-Light)


🚀 LoRA 어댑터 로드 중... (adapter=poketmon/ax-trained-model-v3-2)


Loading checkpoint shards: 100%|██████████| 3/3 [00:06<00:00,  2.08s/it]
Device set to use cuda:0


✅ 모델을 cuda:0 에 올렸습니다.
🚀 pipeline 생성 중...


In [126]:
passage = """
지현: 어… 다 들어왔죠? 오늘은 좀 깁니다. 데이터 레이크 전면 재설계랑 ETL 리워크, 그리고 인프라 케파 다시 짜는 건데요. 요즘 장애가 너무 잦아서 그냥 핫픽스 때려넣고 버티는 수준이 아니라 구조 자체를 갈아엎어야 하는 상황이에요.
하람: 네. 어제 밤에도 레이크 읽기 딜레이 터졌어요. hudi 파일 조각 천 개 넘어서 완전 데이터 블로트 나고, 플링크는 백프레셔 걸려서 ETL이 세 시간씩 밀리고… 그냥 전체가 꽉 막혀 있어요.
민수: AI 쪽도 요즘 콜드 스타트 너무 심해요. 인덱싱이 실시간이 아니고 그냥 어제 데이터 수준이라 RAG라고 부르기도 민망해요. 임베딩은 빨리 되는데 레이크에서 파일 긁어올 때 완전 막혀요.
윤기: 그거 컨테이너 노드에서 I/O 터진 거여서 그래요. 지난번에도 그랬잖아요. executor가 죽었는데, 헬스체크가 제대로 안 돼서 컨테이너 오펀 만드는 바람에 자원만 잡아먹고 있는 상태였어요.
다은: 맞아요. 오펀 생기면 진짜 답 없어요. 스케일아웃도 먹히지도 않고, 워커 노드도 죽은 척 살아있는 애들이 있으니까 autoscaler도 헷갈리고… 결국 ETL 전체가 밀리죠.
지현: 그러면 ETL은 병렬도가 낮아서 밀리는 게 1번이고, 컨테이너 오펀이 케파를 갉아먹는 게 2번이라는 거죠?
하람: 네. 그리고 파티션도 문제예요. 카프카 파티션 400개 넘는데 플링크 parallelism이 32예요. 이건 그냥 스프린트 때 스코프 크리프 난 결과죠.
민수: AI는 또 AI대로 스키마 터진 거 때문에 힘들어요. RAG 인덱싱이 엔드투엔드로 가야 되는데, 레이크에 들어오는 JSON 필드가 매번 달라요. 원래 key-value 몇 개만 쓰던 걸 갑자기 고객사별로 구조가 다르게 들어오니까 모델이 헷갈릴 수밖에 없어요.
소연: QA에서도 계속 문제 생겨요. 같은 쿼리인데 2초 나왔다가 10초 나왔다가… 완전 슬로우쿼리 뽑기운이에요. 안정성이 아예 없어요.
해준: 인프라 입장에서 보면 지금 케파는 이미 끝났어요. CPU는 100 찍고, 디스크 I/O는 울면서 버티고, 레이크 스캔은 하루 종일 풀스캔 찍고 있고… 지금 상태로는 다음 달 캠페인 트래픽 못 받습니다.
지현: 그러면 케파 재설계를 아예 오늘 확정해야겠네요. 더 미룰 수가 없어요.
하람: 우선순위는 레이크 구조 정리, ETL 병렬도 확장, ingestion 스케일 아웃, 인프라 워크로드 분리 순서가 맞아요.
다은: 레이크 쪽은 GC 주기를 늘려야 해요. 지금 delete marker 5일은 너무 짧아서 historical query가 깨져요. 최소 14일로 올려야 해요.
지현: 14일이면 비용 올라가죠?
다은: 네. 근데 지금 성능이 죽어 있는데 비용 아껴봤자 의미 없어요.
민수: RAG 쪽도 이참에 완전 리팩해야 해요. 인덱싱 비용이 지난달 대비 두 배예요. 다 조각 파일 때문이죠.
윤기: 인프라도 워크로드 분리해야죠. 지금은 ETL, 레이크, RAG, 임베딩이 다 같은 클러스터에서 놀고 있으니까 하나 죽으면 다 죽어요.
해준: 케파 잡을 때 레이크는 고정 케파로 가고, ETL은 좀 공격적으로 auto-scaling 걸고, RAG랑 인퍼런스는 트래픽 기준으로 가야 돼요. 지금처럼 한 스택에 다 넣는 방식은 이제 한계예요.
소연: QA 기준도 바꿔야 해요. 지금은 전체 레이턴시만 보는데, ingestion 레이턴시, 레이크 읽기 레이턴시, 인덱싱 레이턴시, RAG 응답 레이턴시 이렇게 쪼개서 봐야 어디서 병목인지 잡혀요.
지현: 좋습니다. 그럼 오늘 목표는 여기까지. 제가 오늘 저녁까지 정리해서 데크 만들고, 내일 스탠드업에서 공유할게요.
민수: 전 인텐트 스키마부터 먼저 잡아볼게요. 요즘 자꾸 꼬여요.
하람: 저희는 compaction tuning 바로 들어갈게요.
다은: 카프카 파티션도 이번 주 안에 손대볼게요.
해준: 인프라는 워크로드 분리 초안 만들어서 내일까지 드릴게요.
지현: 오케이 다들 고생 많았습니다. 회의 종료할게요.
"""

In [127]:
from datetime import date

current_date = date.today().isoformat()

In [128]:
record_id = "pan_text"

In [129]:
summery_prompt_template = """
[시스템]
현재 날짜는 {current_date} 입니다. 이 날짜를 기준으로 'due_date'를 계산하십시오.
'오늘' = {current_date}
'내일' = (오늘 + 1일)
'이번 주 금요일' = (오늘 기준 금요일 날짜)

[지시]
[입력 전문]만을 근거로 "summary", "agendas" 필드를
요청된 **JSON 형식**으로만 출력하라.
**다른 설명이나 텍스트, 마크다운(` ```json `)을 포함하지 말고, 순수한 JSON 객체 하나만 생성해야 한다.**

[입력 전문]
{transcript}

[출력 형식 (JSON)]
아래 JSON 구조를 정확히 따르되, 실제 값은 전문 기반으로 채워라.

{{
  "id": "{record_id}",
    "summary": {{
      "agenda_1": {{
        "who": "...",
        "what": "...",
        "when": "...",
        "where": "...",
        "why": "...",
        "how": "...",
        "how_much": "...",
        "how_long": "..."
      }},
      "agenda_2": {{
        "who": "...",
        "what": "...",
        "when": "...",
        "where": "...",
        "why": "...",
        "how": "...",
        "how_much": "...",
        "how_long": "..."
      }}
    }}
  }},
  "agendas": [
    {{
      "title": "안건 제목",
      "description": "안건 설명 (없으면 null)"
    }}
  ]
}}

[규칙]
1. 반드시 유효한 JSON 객체 하나만 출력한다.
2. id에는 시스템에서 전달된 {record_id}를 그대로 사용한다.
3. summary.agendas와 agendas는 형식 개수 상관 없이 추출할 수 있는대로 뽑는다.
4. summary.agendas:
   - summary.agenda_1, summary.agenda_2 … 의 모든 필드는 '전문에 나오는 단어만' 사용해야 한다.
   - 각 agenda도 5W3H 8개 항목을 포함한다.
   - 전문에 직접 언급되지 않은 표현을 절대 생성하지 말고, 해당 항목은 null 로 설정한다.
5. agendas:
   - 안건 리스트를 title/description 형태로 제공한다.
   - 안건 리스트 없으면 [].
   - description은 없으면 null.
   - summary의 agenda_1, agenda_2… 와 순서를 동일하게 맞춘다.
6. 근거 없는 정보 생성 금지.
7. 출력은 JSON 한 덩어리로. 부가 텍스트 금지.

위 규칙을 모두 지켜 JSON 결과만 출력하라.
"""

In [130]:
task_prompt = """
[시스템]
현재 날짜는 {current_date} 입니다. 이 날짜를 기준으로 'due_date'를 계산하십시오.
'오늘' = {current_date}
'내일' = (오늘 + 1일)
'이번 주 금요일' = (오늘 기준 금요일 날짜)

[지시]
[입력 전문]만을 근거로 "tasks" 필드를
요청된 **JSON 형식**으로만 출력하라.
**다른 설명이나 텍스트, 마크다운(` ```json `)을 포함하지 말고, 순수한 JSON 객체 하나만 생성해야 한다.**

[입력 전문]
{transcript}

{{
  "id": "{record_id}",
  "tasks": [
    {{
      "description": "태스크 내용",
      "assignee": "담당자 이름",
      "due": "마감일 표현",
      "due_date": "YYYY-MM-DD"
    }}
  ]
}}

[규칙]
1. 반드시 유효한 JSON 객체 하나만 출력한다.
2. id에는 시스템에서 전달된 {record_id}를 그대로 사용한다.
3. tasks는 개수 상관없이 모든 태스크를 뽑는다. 
4. tasks를 뽑을 경우, IT 분야에만 국한되지 말고, 기본적인 태스크도 모두 추출한다.
5. tasks:
   - 태스크 리스트 없으면 [].
   - due: 전문에 나온 마감일 표현 (예: "오늘 오후", "다음 주 초"). 없으면 *(언급 없음)*.
   - due_date: [시스템]의 {current_date} 기준 계산. 계산 불가 또는 due 없음 → null.
6. 근거 없는 정보 생성 금지.
7. 출력은 JSON 한 덩어리로. 부가 텍스트 금지.
"""

In [131]:
prompt = task_prompt.format(
    current_date=current_date,
    record_id="syn_pan",
    transcript=passage
)

In [132]:
messages = [
    {"role": "system", "content": prompt},
    {"role": "user", "content": passage}
]

output = pipe(messages, max_new_tokens=1024, temperature=0.1)
prediction = output[0]['generated_text'].strip()
print(prediction)

```json
{
  "id": "syn_pan",
  "tasks": [
    {
      "description": "레이크 구조 정리",
      "assignee": "하람",
      "due": "오늘",
      "due_date": "2025-11-22"
    },
    {
      "description": "ETL 병렬도 확장",
      "assignee": "하람",
      "due": "오늘",
      "due_date": "2025-11-22"
    },
    {
      "description": "ingestion 스케일 아웃",
      "assignee": "하람",
      "due": "오늘",
      "due_date": "2025-11-22"
    },
    {
      "description": "인프라 워크로드 분리",
      "assignee": "해준",
      "due": "오늘",
      "due_date": "2025-11-22"
    },
    {
      "description": "GC 주기 늘리기",
      "assignee": "다은",
      "due": "오늘",
      "due_date": "2025-11-22"
    },
    {
      "description": "RAG 리팩",
      "assignee": "민수",
      "due": "오늘",
      "due_date": "2025-11-22"
    },
    {
      "description": "카프카 파티션 조정",
      "assignee": "다은",
      "due": "이번 주",
      "due_date": "2025-11-28"
    },
    {
      "description": "인텐트 스키마 정리",
      "assignee": "민수",
      "due": "오늘",
      "due_date":

In [133]:
messages = [
    {"role": "system", "content": prompt},
    {"role": "user", "content": passage}
]

base_output = base_pipe(messages, max_new_tokens=1024, temperature=0.1)
base_prediction = base_output[0]['generated_text'].strip()
print(base_prediction)

```json
{
  "id": "syn_pan",
  "tasks": [
    {
      "description": "레이크 구조 정리",
      "assignee": "하람",
      "due": "오늘",
      "due_date": "2025-11-22"
    },
    {
      "description": "ETL 병렬도 확장",
      "assignee": "하람",
      "due": "오늘",
      "due_date": "2025-11-22"
    },
    {
      "description": "Ingestion 스케일 아웃",
      "assignee": "하람",
      "due": "오늘",
      "due_date": "2025-11-22"
    },
    {
      "description": "인프라 워크로드 분리",
      "assignee": "해준",
      "due": "내일",
      "due_date": "2025-11-23"
    },
    {
      "description": "레이크 GC 주기 조정",
      "assignee": "다은",
      "due": "오늘",
      "due_date": "2025-11-22"
    },
    {
      "description": "RAG 인덱싱 리팩",
      "assignee": "민수",
      "due": "오늘",
      "due_date": "2025-11-22"
    },
    {
      "description": "카프카 파티션 조정",
      "assignee": "다은",
      "due": "이번 주",
      "due_date": "2025-11-28"
    },
    {
      "description": "인텐트 스키마 정리",
      "assignee": "민수",
      "due": "오늘",
      "due

In [134]:
refs = [
"""
{{
  "id": "{record_id}",
  "tasks": [
    {{
      "description": "Hudi compacting 튜닝 작업 착수",
      "assignee": "하람",
      "due": "이번 주",
      "due_date": "2025-11-21"
    }},
    {{
      "description": "Kafka 파티션 튜닝 및 재설계 시작",
      "assignee": "다은",
      "due": "이번 주",
      "due_date": "2025-11-21"
    }},
    {{
      "description": "인프라 워크로드 분리 구조 초안 작성",
      "assignee": "해준",
      "due": "내일까지",
      "due_date": "2025-11-21"
    }},
    {{
      "description": "RAG용 인텐트 스키마 정리 및 정규화",
      "assignee": "민수",
      "due": "기한 미정",
      "due_date": null
    }},
    {{
      "description": "오늘 회의 내용을 데크로 정리 및 공유",
      "assignee": "지현",
      "due": "오늘 저녁",
      "due_date": "2025-11-20"
    }}
  ]
}}


"""]

In [135]:
cands = [prediction]

print("🚀 BERTScore 계산 중...")
P, R, F1 = score(cands, refs, lang="ko", verbose=True)
print("\n=== 최종 결과 ===")
print(f"Precision (정밀도): {P.mean().item():.4f}")
print(f"Recall    (재현율): {R.mean().item():.4f}")
print(f"F1 Score  (조화평균): {F1.mean().item():.4f}")

🚀 BERTScore 계산 중...
calculating scores...
computing bert embedding.


100%|██████████| 1/1 [00:00<00:00, 52.92it/s]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 358.40it/s]

done in 0.03 seconds, 32.84 sentences/sec

=== 최종 결과 ===
Precision (정밀도): 0.8476
Recall    (재현율): 0.8314
F1 Score  (조화평균): 0.8394


In [136]:
base_cands = [base_prediction]

print("🚀 BERTScore 계산 중...")
P, R, F1 = score(base_cands, refs, lang="ko", verbose=True)
print("\n=== 최종 결과 ===")
print(f"Precision (정밀도): {P.mean().item():.4f}")
print(f"Recall    (재현율): {R.mean().item():.4f}")
print(f"F1 Score  (조화평균): {F1.mean().item():.4f}")

🚀 BERTScore 계산 중...
calculating scores...
computing bert embedding.


100%|██████████| 1/1 [00:00<00:00, 53.57it/s]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 403.84it/s]

done in 0.03 seconds, 33.99 sentences/sec

=== 최종 결과 ===
Precision (정밀도): 0.8485
Recall    (재현율): 0.8348
F1 Score  (조화평균): 0.8416
